In [22]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt
from faker import Faker

In [2]:
files = os.listdir(path='data')

In [3]:
files

['friday.csv', 'monday.csv', 'thursday.csv', 'tuesday.csv', 'wednesday.csv']

In [4]:
def read_files(files, datapath='data/', delimiter = ';'):
        df_temp = []

        if type(files) == list:
            for i in files:
                df = pd.read_csv(datapath+i, delimiter=delimiter, parse_dates=True, index_col='timestamp')
                day_name = df.index.day_name()[0][:3]
                df['customer_no'] = df.customer_no.apply(lambda x: f'{day_name}_{x}')
                df_temp.append(df)
            df = pd.concat(df_temp)
        else:
            df = pd.read_csv(files, delimiter=delimiter, parse_dates=True, index_col='timestamp')
            day_name = df.index.day_name()[0][:3]
            df['customer_no'] = df.customer_no.apply(lambda x: f'{day_name}_{x}')

        return df

In [5]:
shop_data = read_files(files)

# Part 1: Data Exploration

In [6]:
shop_data = shop_data.sort_values(by='timestamp')

In [7]:
#shop_data = shop_data.reset_index()

In [8]:
shop_data

,customer_no,location
timestamp,,
2019-09-02 07:03:00,Mon_1,dairy
2019-09-02 07:03:00,Mon_2,dairy
2019-09-02 07:04:00,Mon_3,dairy
2019-09-02 07:04:00,Mon_4,dairy
2019-09-02 07:04:00,Mon_5,spices
...,...,...
2019-09-06 21:50:00,Fri_1509,drinks
2019-09-06 21:50:00,Fri_1507,checkout
2019-09-06 21:50:00,Fri_1508,checkout


In [9]:
df_timefill = shop_data.groupby(['customer_no', 'location']).resample('T').ffill()

In [10]:
df_timefill = df_timefill.droplevel('location').reset_index(level=1)

In [11]:
df_timefill = df_timefill.drop('customer_no', axis=1)

In [12]:
df_timefill = df_timefill.reset_index()

In [13]:
customer_flow = pd.pivot_table(df_timefill, index='customer_no', columns='location', values='timestamp' )

In [14]:
customer_flow.apply(np.sort, axis=1)

customer_no
Fri_1       [2019-09-06T07:00:00.000000000, 2019-09-06T07:...
Fri_10      [2019-09-06T07:06:00.000000000, 2019-09-06T07:...
Fri_100     [2019-09-06T08:03:00.000000000, 2019-09-06T08:...
Fri_1000    [2019-09-06T17:19:00.000000000, 2019-09-06T17:...
Fri_1001    [2019-09-06T17:22:30.000000000, 2019-09-06T17:...
                                  ...                        
Wed_995     [2019-09-04T16:52:00.000000000, 2019-09-04T16:...
Wed_996     [2019-09-04T16:52:00.000000000, 2019-09-04T16:...
Wed_997     [2019-09-04T16:53:00.000000000, 2019-09-04T17:...
Wed_998     [2019-09-04T16:53:00.000000000, 2019-09-04T16:...
Wed_999     [2019-09-04T16:53:00.000000000, 2019-09-04T16:...
Length: 7445, dtype: object

# Create entrance times and fill checkout times

In [15]:
def fill_enter(dataframe):
    
    entry_time = dataframe.loc[dataframe.notnull()].min() - dt.timedelta(minutes=1)
    return entry_time
    #if dataframe.entrance != dataframe.entrance:
    #    entry_time = dataframe.loc[dataframe.notnull()].min() - timedelta(minuntes=1)
    #    return entry_time
    #elif dataframe.entrance:
    #    return dataframe.entrance

In [16]:
def fill_exit(dataframe):
    if dataframe.checkout != dataframe.checkout:
        closing_time = dataframe.loc[dataframe.notnull()].max().round('H')
        return closing_time
    elif dataframe.checkout:
        return dataframe.checkout

In [17]:
customer_flow['entrance'] = customer_flow.apply(fill_enter, axis=1)
customer_flow['checkout'] = customer_flow.apply(fill_exit, axis=1)

In [18]:
customer_flow = customer_flow.reset_index(level=0)

In [19]:
customer_flow = customer_flow.melt(id_vars='customer_no', value_name='timestamp')

In [20]:
customer_flow = customer_flow.dropna()

# Make classes

In [38]:
fake = Faker()

In [51]:
class Store():
    def __init__(self):
        self.customers = []
        self.customer_count = len(self.customers)
        
    def store_customer(self, data):
        self.customers.append(data)

In [101]:
class Customer():
    
    def __init__(self, name, cust_id, locations):
        self.name = name
        self.cust_id = cust_id
        self.customer_flow = locations
        self.data = {self.cust_id: (self.name, self.customer_flow)}
        #self.probability = probability
        #self.time_in_store = time_in_store
        
    def send_info(self):
        return self.data
    
    def time_in_store(self):
        pass
        #time_in_store
        
    def get_flow_probability(self):
        #print(self.customer_flow)
        self.flow_shift = pd.DataFrame(self.customer_flow)
        self.flow_shift['loc_shift'] = self.flow_shift.shift(-1)
        self.prob = self.flow_shift
        self.probability = pd.crosstab(self.prob[0], self.prob['loc_shift'], normalize=0)
        print(self.probability)

In [34]:
customer_flow = customer_flow.sort_values(by=['customer_no','timestamp'])

In [36]:
location_list = customer_flow.groupby('customer_no')['location'].apply(list)

In [102]:
food_store = Store()

for i in location_list.index:
    #print(i, location_list[i])
    cust = Customer(fake.name(), i, location_list[i], )
    data = cust.send_info()
    food_store.store_customer(data)
    cust.get_flow_probability()

loc_shift  checkout  dairy  spices
0                                 
dairy           0.0    0.0     1.0
entrance        0.0    1.0     0.0
spices          1.0    0.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  drinks  fruit  spices
0                                         
drinks          0.0     0.0    1.0     0.0
entrance        0.0     1.0    0.0     0.0
fruit           0.0     0.0    0.0     1.0
spices          1.0     0.0    0.0     0.0
loc_shift  checkout  dairy  fruit
0                                
dairy           0.0    0.0    1.0
entrance        0.0    1.0    0.0
fruit           1.0    0.0    0.0
loc_shift  checkout  dairy
0                         
dairy           1.0

loc_shift  checkout  drinks  fruit
0                                 
drinks          0.0     0.0    1.0
entrance        0.0     1.0    0.0
fruit           1.0     0.0    0.0
loc_shift  checkout  dairy  drinks  fruit
0                                        
dairy           0.0    0.0     1.0    0.0
drinks          1.0    0.0     0.0    0.0
entrance        0.0    0.0     0.0    1.0
fruit           0.0    1.0     0.0    0.0
loc_shift  checkout  dairy  fruit
0                                
dairy           0.0    0.0    1.0
entrance        0.0    1.0    0.0
fruit           1.0    0.0    0.0
loc_shift  checkout  drinks  fruit
0                                 
drinks          1.0     0.0    0.0
entrance        0.0     0.0    1.0
fruit           0.0     1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  c

loc_shift  checkout  dairy  drinks  spices
0                                         
dairy           0.0    0.0     1.0     0.0
drinks          1.0    0.0     0.0     0.0
entrance        0.0    0.0     0.0     1.0
spices          0.0    1.0     0.0     0.0
loc_shift  checkout  dairy  drinks
0                                 
dairy           0.0    0.0     1.0
drinks          1.0    0.0     0.0
entrance        0.0    1.0     0.0
loc_shift  checkout  dairy
0                         
dairy           1.0    0.0
entrance        0.0    1.0
loc_shift  checkout  dairy  drinks  fruit
0                                        
dairy           0.0    0.0     0.0    1.0
drinks          0.0    1.0     0.0    0.0
entrance        0.0    0.0     1.0    0.0
fruit           1.0    0.0     0.0    0.0
loc_shift  checkout  spices
0                          
entrance        0.0     1.0
spices          1.0     0.0
loc_shift  checkout  spices
0                          
entrance        0.0     1.0
spices     

loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           0.0    0.0    1.0     0.0
entrance        0.0    0.0    0.0     1.0
fruit           1.0    0.0    0.0     0.0
spices          0.0    1.0    0.0     0.0
loc_shift  checkout  fruit  spices
0                                 
entrance        0.0    1.0     0.0
fruit           0.0    0.0     1.0
spices          1.0    0.0     0.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           0.0    0.0    0.0     1.0
entrance        0.0    0.0    1.0     0.0
fruit           0.0    1.0    0.0     0.0
spices          1.0    0.0    0.0     0.0
loc_shift  checkout  dairy  spices
0                                 
dairy           0.0    0.0     1.0
entrance        0.0    1.0     0.0
spices          1.0    0.0     0.0
loc_shift  checkout  dairy  drinks  fruit  spices
0                                                
dairy           0.0    0.0     0.0    0.0     

loc_shift  checkout  drinks  fruit  spices
0                                         
drinks          0.0     0.0    0.0     1.0
entrance        0.0     1.0    0.0     0.0
fruit           1.0     0.0    0.0     0.0
spices          0.0     0.0    1.0     0.0
loc_shift  checkout  drinks  fruit  spices
0                                         
drinks          0.0     0.0    0.0     1.0
entrance        0.0     0.0    1.0     0.0
fruit           0.0     1.0    0.0     0.0
spices          1.0     0.0    0.0     0.0
loc_shift  checkout  drinks  spices
0                                  
drinks          1.0     0.0     0.0
entrance        0.0     0.0     1.0
spices          0.0     1.0     0.0
loc_shift  checkout  dairy  drinks  fruit  spices
0                                                
dairy           0.0    0.0     1.0    0.0     0.0
drinks          0.0    0.0     0.0    0.0     1.0
entrance        0.0    1.0     0.0    0.0     0.0
fruit           1.0    0.0     0.0    0.0     0.0
spic

...

loc_shift  checkout  drinks
0                          
drinks          1.0     0.0
entrance        0.0     1.0
loc_shift  checkout  dairy  drinks  fruit
0                                        
dairy           0.0    0.0     1.0    0.0
drinks          1.0    0.0     0.0    0.0
entrance        0.0    0.0     0.0    1.0
fruit           0.0    1.0     0.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit  spices
0                                 
entrance        0.0    0.0     1.0
fruit           1.0    0.0     0.0
spices          0.0    1.0     0.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           0.0    0.0    0.0     1.0
entrance        0.0    0.0    1.0     0.0
fruit           0.0    1.0    0.0     0.0
spices          1.0    0.0    0.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0 

loc_shift  checkout  drinks  fruit  spices
0                                         
drinks          1.0     0.0    0.0     0.0
entrance        0.0     0.0    0.0     1.0
fruit           0.0     1.0    0.0     0.0
spices          0.0     0.0    1.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  drinks  spices
0                                  
drinks          1.0     0.0     0.0
entrance        0.0     0.0     1.0
spices          0.0     1.0     0.0
loc_shift  checkout  dairy
0                         
dairy           1.0    0.0
entrance        0.0    1.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           0.0    0.0    0.0     1.0
entrance        0.0    1.0    0.0     0.0
fruit           1.0    0.0    0.0     0.0
spices          0.0    0.0    1.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit        

loc_shift  checkout  dairy  fruit
0                                
dairy           0.0    0.0    1.0
entrance        0.0    1.0    0.0
fruit           1.0    0.0    0.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           1.0    0.0    0.0     0.0
entrance        0.0    0.0    0.0     1.0
fruit           0.0    1.0    0.0     0.0
spices          0.0    0.0    1.0     0.0
loc_shift  checkout  dairy  fruit
0                                
dairy           0.0    0.0    1.0
entrance        0.0    1.0    0.0
fruit           1.0    0.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  spices
0                          
entrance        0.0     1.0
spices          1.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0

loc_shift  checkout  dairy  drinks  fruit  spices
0                                                
dairy           1.0    0.0     0.0    0.0     0.0
drinks          0.0    0.0     0.0    1.0     0.0
entrance        0.0    0.0     0.0    0.0     1.0
fruit           0.0    1.0     0.0    0.0     0.0
spices          0.0    0.0     1.0    0.0     0.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           0.0    0.0    1.0     0.0
entrance        0.0    1.0    0.0     0.0
fruit           0.0    0.0    0.0     1.0
spices          1.0    0.0    0.0     0.0
loc_shift  checkout  spices
0                          
entrance        0.0     1.0
spices          1.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  spices
0                          
entrance        0.0     1.0
spices          1.0     0.0
loc_shift  checkout  fruit
0                         
entrance    

entrance        0.0     1.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           0.0    0.0    1.0     0.0
entrance        0.0    0.0    0.0     1.0
fruit           1.0    0.0    0.0     0.0
spices          0.0    1.0    0.0     0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  fruit
0                         
entrance        0.0    1.0
fruit           1.0    0.0
loc_shift  checkout  dairy  fruit  spices
0                                        
dairy           1.0    0.0    0.0     0.0
entrance        0.0    0.0    0.0     1.0
fruit           0.0    1.0    0.0     0.0
spices          0.0    0.0    1.0     0.0
loc_shift  checkout  dairy  drinks
0                                 
dairy           0.0    0.0     1.0
drinks          1.0    0.0     0.0
entr

# Shift location one step

AttributeError: 'Store' object has no attribute 'cust'

In [186]:
customer_transitions = customer_flow.sort_values(by=['customer_no', 'timestamp'])

In [187]:
customer_transitions = customer_transitions[['customer_no', 'location']]

In [188]:
customer_transitions['location+1'] = customer_transitions.groupby('customer_no')['location'].shift(-1)

In [189]:
customer_transitions

,customer_no,location,location+1
37225,Fri_1,entrance,dairy
7445,Fri_1,dairy,spices
29780,Fri_1,spices,checkout
0,Fri_1,checkout,NaN
37226,Fri_10,entrance,fruit
...,...,...,...
29778,Wed_998,fruit,checkout
7443,Wed_998,checkout,NaN
44669,Wed_999,entrance,fruit
29779,Wed_999,fruit,checkout


# Create Probabilities table

In [190]:
probabilites = pd.crosstab(customer_transitions['location'], customer_transitions['location+1'], normalize=0)

In [191]:
probabilites

location+1,checkout,dairy,drinks,fruit,spices
location,,,,,
dairy,0.452069,0.000000,0.202200,0.174175,0.171556
drinks,0.625078,0.054449,0.000000,0.165215,0.155258
entrance,0.000000,0.305440,0.164540,0.357018,0.173002
fruit,0.587302,0.173669,0.123249,0.000000,0.115780
spices,0.406399,0.212730,0.234513,0.146358,0.000000


# Predict states

In [192]:
from random import choices

In [197]:
location_states = list(customer_transitions.location.unique())
location_states

['entrance', 'dairy', 'spices', 'checkout', 'fruit', 'drinks']